In [ ]:
#Парсинг коммерческих объектов с avito

In [1]:
#Импортируем необходиимых библиотек для реализации парсинга
from selenium import webdriver
from selenium_stealth import stealth # делает похожим на настоящего пользователя
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from time import sleep
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("start_maximized")

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

link_all = []

p = 26 # количество страниц

for page in tqdm(range(1, p, 1), 'Обработанно страниц'):
    
    url = f"https://www.avito.ru/ekaterinburg/kommercheskaya_nedvizhimost/prodam-ASgBAgICAUSwCNJW?f=ASgBAQICAkSwCNJW8hKg2gEBQJ7DDTSI2TmG2TmE2Tk&p={page}"
    driver.get(url)
    sleep(2) #спим 2 секунды (прогрузка страницы)
    
    #выгружаем ссылку по всем квартирам
    blocks = driver.find_elements_by_class_name('iva-item-titleStep-pdebR')
    for block in blocks:
        link = block.find_element_by_tag_name('a').get_attribute("href")
        link_all.append(link)

C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\2446538235.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")
Обработанно страниц:   0%|                                                                      | 0/25 [00:00<?, ?it/s]C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\2446538235.py:27: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  blocks = driver.find_elements_by_class_name('iva-item-titleStep-pdebR')
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\2446538235.py:29: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  link = block.find_element_by_tag_name('a').get_attribute("href")
Обработанно страниц: 100%|█████████████████████████████████████████████████████████████| 25/25 [02:31<00:00,  

In [3]:
len(link_all)

1205

In [4]:
#Выгружаем данные (характеристики, адрес, текст объявления)
t1 = [] #список
a = 0

for lnk in tqdm(link_all, 'Сбор данных'):
    try:
        a = a + 1 #счетчик
        t1.clear() #очищаем список значений
        
        driver.get(lnk) #открываем ссылку
        sleep(2) #спим 2 секунды (прогрузка страницы)
        
        #ТАБЛИЦА 1
        table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
        k_1 = len(table_1) #количество строк в таблице 1

        for j in range(0, k_1):
            table_t1 = table_1[j].text.split(':') #выгружаем текст и разделяем его на 2 части по ":"
            t1.append(table_t1) #складываем в список
        
        #ТАБЛИЦА 2
        table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H').find_elements_by_tag_name('li') #таблица 2
        k_2 = len(table_2) #количество строк в таблице 2

        for j in range(0, k_2):
            table_t2 = table_2[j].text.split(':') #выгружаем текст и разделяем его на 2 части по ":"
            t1.append(table_t2) #складываем в список
        
        #АДРЕС
        address = driver.find_element_by_class_name('style-item-address-KooqC').find_element_by_tag_name('span').text #адрес
        addr = ['Адрес', address] #создаем список для адреса
        t1.append(addr) #складываем в список
        
        #ТЕКСТ ОБЪЯВЛЕНИЯ
        text_o = driver.find_element_by_class_name('style-item-description-pL_gy').text #текст объявления
        to = ['Текст', text_o] #создаем список для адреса
        t1.append(to) #складываем в список
         
        #ЦЕНА
        price = driver.find_element_by_class_name('style-price-value-main-TIg6u').find_element_by_tag_name('span').get_attribute("content")
        price_1 = ['Цена', price] #создаем список для цены
        t1.append(price_1) #складываем в список
        
        #ССЫЛКА
        lnk_1 = ['Ссылка', lnk] #создаем список для ссылки
        t1.append(lnk_1) #складываем в список
        
        df = pd.DataFrame(t1)
        df = df.set_index(0)    

        if a == 1:
            df_result = df
        if a > 1:
            df_result = pd.concat([df_result, df], axis=1)
    
    except:
        continue

df_result_copy = df_result.copy()
df_result_copy = df_result_copy.transpose()

#Сохраняем в Excel
writer = pd.ExcelWriter('Квартиры_Тула_перв_2.xlsx')
df_result_copy.to_excel(writer, 'marks')
writer.save()

Сбор данных:   0%|                                                                            | 0/1205 [00:00<?, ?it/s]C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:22: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H

Сбор данных:   0%|▏                                                                 | 4/1205 [00:15<1:12:10,  3.61s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:22: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H

Сбор данных:   1%|▍                                                                 | 7/1205 [00:25<1:08:44,  3.44s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:22: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H

Сбор данных:   1%|▌                                                                | 10/1205 [00:35<1:08:44,  3.45s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:22: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H

Сбор данных:   1%|▋                                                                | 13/1205 [00:46<1:12:29,  3.65s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:22: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H

...

Сбор данных:  99%|██████████████████████████████████████████████████████████████▎| 1192/1205 [1:14:51<00:42,  3.25s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:22: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H

Сбор данных:  99%|██████████████████████████████████████████████████████████████▍| 1195/1205 [1:15:01<00:33,  3.31s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:22: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H

Сбор данных:  99%|██████████████████████████████████████████████████████████████▋| 1198/1205 [1:15:11<00:22,  3.25s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:22: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H

Сбор данных: 100%|██████████████████████████████████████████████████████████████▊| 1201/1205 [1:15:21<00:13,  3.25s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:22: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H

Сбор данных: 100%|██████████████████████████████████████████████████████████████▉| 1204/1205 [1:15:31<00:03,  3.28s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:14: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_11424\1929709951.py:22: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H